In [ ]:
!pip install pytorch-lightning
!pip install torchvision
!pip install torchmetrics

In [ ]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
import torchmetrics

In [ ]:
dataset = MNIST(os.getcwd(), download=True, transform=transforms.ToTensor())

train_set, val_set, test_set, other = torch.utils.data.random_split(dataset, [5000, 1000, 1000, 53000])
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 81186704.25it/s]


Extracting /content/MNIST/raw/train-images-idx3-ubyte.gz to /content/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 85007504.44it/s]


Extracting /content/MNIST/raw/train-labels-idx1-ubyte.gz to /content/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 21765813.66it/s]


Extracting /content/MNIST/raw/t10k-images-idx3-ubyte.gz to /content/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 17271558.27it/s]


Extracting /content/MNIST/raw/t10k-labels-idx1-ubyte.gz to /content/MNIST/raw



In [ ]:
class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(nn.Linear(28 * 28, 64), nn.ReLU(),
                                nn.Linear(64, 128), nn.ReLU(),
                                nn.Linear(128, 10))

    def forward(self, x):
        return self.l1(x)

In [ ]:
class SimpleTraining(pl.LightningModule):
    def __init__(self, simplenet):
        super().__init__()
        self.simplenet = simplenet
        self.loss = nn.CrossEntropyLoss()
        self.train_accuracy = torchmetrics.Accuracy(task='multiclass', num_classes=10)

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, y = batch
        x = x.view(x.size(0), -1) # Spłaszczamy obraz
        y_hat = self.simplenet(x)
        J = self.loss(y_hat, y)
        self.train_accuracy(y_hat, y)

        self.log('train_loss', J, on_step=False, on_epoch=True)
        self.log('train_acc_step', self.train_accuracy, on_step=False, on_epoch=True)
        return J

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [ ]:
%load_ext tensorboard
%tensorboard --logdir tb_logs/

<IPython.core.display.Javascript object>

In [ ]:
# model
simple_training = SimpleTraining(SimpleNet())

# Configure tensorboard to monitor training

tensorboard = TensorBoardLogger("tb_logs", name="simple_model")
# train model
trainer = pl.Trainer(logger=tensorboard, max_epochs=100)
trainer.fit(model=simple_training, train_dataloaders=train_loader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name           | Type               | Params
------------------------------------------------------
0 | simplenet      | SimpleNet          | 59.9 K
1 | loss           | CrossEntropyLoss   | 0     
2 | train_accuracy | MulticlassAccuracy | 0     
------------------------------------------------------
59.9 K    Trainable params
0         Non-trainable params
59.9 K    Total params
0.239     Total estimated model params size (MB)
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing 

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


In [ ]:
print(SimpleNet())

SimpleNet(
  (l1): Sequential(
    (0): Linear(in_features=784, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=10, bias=True)
  )
)


# Zadania

1. W pętli treningowej brakuje bardzo ważnego elementu - walidacji modelu. Utwórz poniżej dataloader dla zbioru walidacyjnego oraz stwórz nową wersję simple training z metodą do walidacji. Utwórz funkcję błędu oraz metryki dla zbioru walidacyjnego i loguj osobno wyniki dla walidacji. Wyświetl wyniki w tensorboard. Opisz różnicę między wynikami dla zbioru treningowego i walidacyjnego.

Checklist:
- [ ] validation dataloader
- [ ] funkcja błędu dla walidacji
- [ ] metoda do walidacji w lightning module
- [ ] metryka dla walidacji
- [ ] logowanie dla walidacji
- [ ] tensorboard
- [ ] komentarz do wyników

In [ ]:
# utwórz dataloader do walidacji
val_dataloader = DataLoader(val_set, batch_size=32, shuffle=False)

In [ ]:
class SimpleTrainingWithValidation(pl.LightningModule):
    def __init__(self, simplenet):
        super().__init__()
        self.simplenet = simplenet
        self.loss = nn.CrossEntropyLoss()
        self.train_acc = torchmetrics.Accuracy(num_classes=10, task='multiclass')
        self.val_acc = torchmetrics.Accuracy(num_classes=10, task='multiclass')

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, y = batch
        x = x.view(x.size(0), -1) # Spłaszczamy obraz
        y_hat = self.simplenet(x)
        J = self.loss(y_hat, y)
        self.train_acc(y_hat, y)

        self.log('train_loss', J, on_step=False, on_epoch=True)
        self.log('train_acc', self.train_acc, on_step=False, on_epoch=True)
        return J

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

# Walidacja    def ...
    def validation_step(self, batch, batch_idx):
        x, y = batch
        x = x.view(x.size(0), -1)
        y_hat = self.simplenet(x)
        J = self.loss(y_hat, y)
        self.val_acc(y_hat, y)

        self.log('val_loss', J, on_step=False, on_epoch=True)
        self.log('val_acc', self.val_acc, on_step=False, on_epoch=True)


In [ ]:
logger = TensorBoardLogger("logs/", name="model")

model = SimpleTrainingWithValidation(SimpleNet())

trainer = pl.Trainer(max_epochs=5, logger=logger)

trainer.fit(model, train_loader, val_dataloader)

%tensorboard --logdir logs --port 6007

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type               | Params
-------------------------------------------------
0 | simplenet | SimpleNet          | 59.9 K
1 | loss      | CrossEntropyLoss   | 0     
2 | train_acc | MulticlassAccuracy | 0     
3 | val_acc   | MulticlassAccuracy | 0     
-------------------------------------------------
59.9 K    Trainable params
0         Non-trainable params
59.9 K    Total params
0.239     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


Reusing TensorBoard on port 6007 (pid 42132), started 0:06:36 ago. (Use '!kill 42132' to kill it.)

<IPython.core.display.Javascript object>